# Preparação do ambiente

## Bibliotecas

In [203]:
import pandas as pd
import warnings

from datetime import datetime
from pathlib import Path

In [91]:
pd.options.mode.chained_assignment = None  # default='warn'
# Suppress FutureWarning messages
warnings.simplefilter(action='ignore', category=FutureWarning)

## Constantes e funções auxiliares

In [123]:
def format_issue(row):

    descricao_produto = row['descricao_produto']
    arquivo_pagina = row['arquivo_pagina']
    url = row['url']
    
    qual_codigo_sch_fornecido = row['qual_codigo_sch_fornecido']
    info_fabricante = row['info_fabricante']
    info_tipo_produto = row['info_tipo_produto']
    info_modelos_certificado = row['info_modelos_certificado']
    info_ean_homologados = row['info_ean_homologados']
    info_modelo_ean = row['info_modelo_ean']
    

    pdf_folder = 'https://anatel365.sharepoint.com/:b:/r/sites/Desenvolvimentodeappfiscalizaoe-commerce/Documentos%20Compartilhados/General/Resultados/screenshots'

    issue_mask = f'''h1. {descricao_produto}
    
"Aquivo PDF do anúncio coletado":{pdf_folder}/{arquivo_pagina}

---
    
h3. Informações do certificado de homologação:
    
*_Número de Homologação_*: {qual_codigo_sch_fornecido}
    
*_Nome do Fabricante_*: {info_fabricante}
    
*_Tipo do Produto_*: {info_tipo_produto}
    
*_Modelos do Certificado_*: {info_modelos_certificado}

---

h3. Informações do EAN:
   
*_EAN homologados_*: 
* {info_ean_homologados}

*_Modelos do EAN_*: 
* {info_modelo_ean}'''

    return issue_mask

In [207]:
BASE_FOLDER = r'D:\Users\maxwelfreitas\ANATEL\REGULATRON - Desenvolvimento de app fiscalização e-commerce - Documentos\General'

# Carga e prepação dos dados

## Base EAN homologados

In [184]:
file_ean = '../datasets/sch/lista_celulares_homologados_ean.xlsx'
df_ean = pd.concat([df for df in pd.read_excel(file_ean, sheet_name=None,dtype='str').values()])
df_ean = df_ean.dropna()

columns_to_keep = ['Número de Homologação','Código EAN', 'Modelo', 'Nome Comercial']
df_ean = df_ean[columns_to_keep].drop_duplicates()

df_ean['Código EAN'] = df_ean['Código EAN'].apply(lambda x: x.strip())
df_ean['Número de Homologação'] = df_ean['Número de Homologação'].apply(lambda x: x.zfill(12))

df_ean = df_ean.sort_values(by=['Número de Homologação', 'Código EAN'])

columns_to_keep = ['Modelo', 'Nome Comercial']
# df_ean['Modelo Completo'] = df_ean[columns_to_keep].apply(lambda row: ' | '.join(row),axis=1)
df_ean['Modelo Completo'] = df_ean[columns_to_keep].apply(lambda row: ': '.join(row),axis=1)

columns_to_keep = ['Número de Homologação', 'Código EAN']
df_sch_ean = df_ean[columns_to_keep].drop_duplicates()
# df_sch_ean['Lista EAN'] = df_sch_ean[columns_to_keep].groupby('Número de Homologação').transform(lambda x : ' | '.join(x))
df_sch_ean['Lista EAN'] = df_sch_ean[columns_to_keep].groupby('Número de Homologação').transform(lambda x : '\n* '.join(x))
df_sch_ean = df_sch_ean.drop_duplicates(subset='Número de Homologação')
df_sch_ean = df_sch_ean[['Número de Homologação', 'Lista EAN']]


columns_to_keep = ['Número de Homologação', 'Modelo Completo']
df_sch_ean_modelo = df_ean[columns_to_keep].drop_duplicates()
# df_sch_ean_modelo['Lista Modelo'] = df_sch_ean_modelo[columns_to_keep].groupby('Número de Homologação').transform(lambda x : ' | '.join(x))
df_sch_ean_modelo['Lista Modelo'] = df_sch_ean_modelo[columns_to_keep].groupby('Número de Homologação').transform(lambda x : '\n* '.join(x))
df_sch_ean_modelo = df_sch_ean_modelo.drop_duplicates(subset='Número de Homologação')

columns_to_keep = ['Número de Homologação', 'Lista Modelo']
df_sch_ean = df_sch_ean.merge(df_sch_ean_modelo[columns_to_keep])
df_sch_ean = df_sch_ean.reset_index(drop=True)

df_sch_ean.columns = ['qual_codigo_sch_fornecido', 'info_ean_homologados', 'info_modelo_ean']

df_sch_ean.head()

,qual_codigo_sch_fornecido,info_ean_homologados,info_modelo_ean
0,000032309185,7908426307013\n* 7908426307020\n* 790842630703...,23021RAAEG: Redmi Note 12\n* 23028RA60L: Redmi...
1,000512201993,0194252145517\n* 0194252145524\n* 019425214585...,A2783: iPhone SE
2,000672100953,7892509117791\n* 7892509117807\n* 789250911781...,SM-A725M/DS: A72
3,001472314550,7908426304814\n* 7908426304821\n* 790842630587...,2201123G: POCO X5 PRO 5G\n* 22101320G: POCO X5...
4,001641811035,7898590571986,HT-705 G: HT-705 3G Kids\n* HT-705 G: HT-705G ...


## Base SCH

In [129]:
%%time

update_homologados = False

if update_homologados:
    
    file_sch = '../datasets/sch/produtos_certificados.zip'
    file_cel_5g = '../datasets/sch/celulares_5g_homologados.zip'
    
    dtype_sch = {'Número de Homologação': 'str'}
    dtype_cel_5g = {'NumeroHomologacao': 'str'}
    
    sch_columns_to_keep = ['Número de Homologação', 'Nome do Fabricante', 'Tipo do Produto', 'Modelo', 'Nome Comercial']
    cel_5g_columns_to_keep = ['NumeroHomologacao', 'Fabricante', 'TipodeProduto', 'Modelo', 'NomeComercial'] 
    
    df_sch = pd.read_csv(file_sch, sep=';',dtype=dtype_sch)
    df_sch = df_sch[sch_columns_to_keep]
    
    df_cel_5g = pd.read_csv(file_cel_5g, sep=';',dtype=dtype_cel_5g)
    df_cel_5g = df_cel_5g[cel_5g_columns_to_keep]
    df_cel_5g.columns = sch_columns_to_keep
    
    df_sch = pd.concat([df_sch, df_cel_5g])
    df_sch = df_sch.drop_duplicates()
    df_sch = df_sch.fillna('')
    
    df_tipo_produto = df_sch[['Número de Homologação','Tipo do Produto']].drop_duplicates()
    df_tipo_produto['Tipo Consolidado'] = df_tipo_produto.groupby('Número de Homologação').transform(lambda x : ' | '.join(x))
    df_tipo_produto = df_tipo_produto[['Número de Homologação', 'Tipo Consolidado']].drop_duplicates()
    df_tipo_produto.columns = ['Número de Homologação', 'Tipo do Produto']
    
    df_modelo = df_sch[['Número de Homologação','Modelo']].drop_duplicates()
    df_modelo['Modelo Consolidado'] = df_modelo.groupby('Número de Homologação').transform(lambda x : ' | '.join(x))
    df_modelo = df_modelo[['Número de Homologação', 'Modelo Consolidado']].drop_duplicates()
    df_modelo.columns = ['Número de Homologação', 'Modelo']
    
    df_nome_comercial = df_sch[['Número de Homologação','Nome Comercial']].drop_duplicates()
    df_nome_comercial['Nome Comercial Consolidado'] = df_nome_comercial.groupby('Número de Homologação').transform(lambda x : ' | '.join(x))
    df_nome_comercial = df_nome_comercial[['Número de Homologação', 'Nome Comercial Consolidado']].drop_duplicates()
    df_nome_comercial.columns = ['Número de Homologação', 'Nome Comercial']
    
    df_modelo_completo = df_modelo.merge(df_nome_comercial,how='left')
    df_modelo_completo['Modelos do Certificado'] = df_modelo_completo[['Modelo','Nome Comercial']].apply(lambda row: ' | '.join(row),axis=1)
    
    df_homologados = df_sch[['Número de Homologação','Nome do Fabricante']].drop_duplicates()
    df_homologados = df_homologados.merge(df_tipo_produto,how='left')
    df_homologados = df_homologados.merge(df_modelo_completo,how='left')
    
    columns_to_keep = ['Número de Homologação', 'Nome do Fabricante', 'Tipo do Produto', 'Modelos do Certificado']
    df_homologados = df_homologados[columns_to_keep]
    
    df_homologados.columns = ['qual_codigo_sch_fornecido', 'info_fabricante', 'info_tipo_produto', 'info_modelos_certificado']

else:
    df_homologados = pd.read_parquet('../datasets/base_sch_consolidada.parquet')

df_homologados.head()

CPU times: total: 15.6 ms
Wall time: 91.3 ms


,qual_codigo_sch_fornecido,info_fabricante,info_tipo_produto,info_modelos_certificado
0,017041004912,Ceragon Networks s.r.o.,Transceptor Digital,Evolution 38GHz | Evolution 38GHz
1,039581206807,Hytera Communications Corporation Ltd.,Transceptor Troncalizado - Móvel,MD786G U(3) | MD786G U(3)
2,041151303428,Commscope,Antena Ponto a Ponto,SHPX2-13 | SHPX3-13 |
3,000580300450,"GE MDS, LLC",Transceptor Digital,LEDR 400F | LEDR 400F
4,027631107312,Soma Investimentos - Serviço de Voz Digital Ltda.,Equipamento para Telecomunicações com interfac...,Pligg | Pligg


## Resultados da raspagem dos marketplaces

In [130]:
map_columns = {
    'Item': 'item',
    'Texto da Busca' : 'texto_busca',
    'Arquivo da Página' : 'arquivo_pagina',
    'Data da Análise' : 'data_analise',
    'Endereço eletrônico (URL)' : 'url',
    'Descrição do produto no anúncio' : 'descricao_produto',
    'O tipo do produto é de uso proibido no Brasil?' : 'produto_proibido',
    'O produto é passível de homologação na Anatel? (Sim ou Não)' : 'produto_passivel',
    'Tipo do Produto' : 'tipo_produto',
    'Fabricante' : 'fabricante',
    'Modelo do Produto' : 'modelo',
    'Valor do Produto (R$)' : 'valor',
    'Número de Unidades à Venda' : 'unidades_venda',
    'Existe o campo código de homologação no anúncio? (Sim ou Não)' : 'existe_campo_codigo_sch',
    'O código de homologação foi fornecido? (Sim ou Não)' : 'codigo_sch_foi_fornecido',
    'O produto é homologado? (Sim, Não e N.A.)' : 'produto_homologado',
    'Validação do código de homologação - O código de homologação fornecido é o do produto anunciado? (Sim, Não, N.A.)' : 'codigo_sch_corresponde_produto',
    'O código EAN foi fornecido? (Sim ou Não) - Apenas para Smartphones.' : 'codigo_ean_foi_fornecido',
    'O código EAN fornecido corresponde ao produto? (Sim ou Não) -  Apenas para Smartphones.' : 'codigo_ean_corresponde_produto',
    'Observação 1 - Qual Código de Homologação fornecido no anúncio?' : 'qual_codigo_sch_fornecido',
    'Marketplace' : 'marketplace'
}

In [208]:
results_folder = Path(BASE_FOLDER)/'Resultados'
result_files = [file for file in base_path.rglob('*.xlsx') if '2024' in file.name]

map_marketplace = {
    'amazon': 'Amazon', 
    'ml': 'Mercado Livre',
    'carrefour': 'Carrefour', 
    'magalu': 'Magazine Luiza'
}

scrape_results = []

for file in result_files:
    
    file_name = file.name
    marketplace_code = file_name.split('_')[0]
    marketplace = map_marketplace[marketplace_code]

    df = pd.read_excel(file,dtype='str')
    df['Marketplace'] = marketplace
    scrape_results.append(df)

df_produtos = pd.concat(scrape_results)
df_produtos.columns = df_produtos.columns.map(map_columns)
# add new columns
df_produtos['qual_codigo_ean_fornecido'] = None

columns_to_keep = ['tipo_produto', 'texto_busca', 'data_analise', 'marketplace',
                   'arquivo_pagina', 'url', 'descricao_produto',
                   'fabricante', 'modelo', 'valor', 
                   'existe_campo_codigo_sch', 'codigo_sch_foi_fornecido', 'qual_codigo_sch_fornecido', 'codigo_sch_corresponde_produto',
                   'codigo_ean_foi_fornecido', 'qual_codigo_ean_fornecido', 'codigo_ean_corresponde_produto']
df_produtos = df_produtos[columns_to_keep]


df_produtos = df_produtos.merge(df_homologados,on='qual_codigo_sch_fornecido',how='left')
df_produtos = df_produtos.merge(df_sch_ean,on='qual_codigo_sch_fornecido',how='left')
df_produtos['info_ean_homologados'].fillna('EAN indisponível',inplace=True)
df_produtos['info_modelo_ean'].fillna('EAN indisponível',inplace=True)
df_produtos = df_produtos.fillna('')
df_produtos['valor'] = df_produtos['valor'].apply(lambda x: 0 if x=='' else x).astype(float)

# strip pipe caracter from the end of string
for col in ['info_tipo_produto', 'info_modelos_certificado', 'info_ean_homologados']:
    df_produtos[col] = df_produtos[col].apply(lambda x: x[:-3] if x[-3:]==' | ' else x)

df_produtos.head()

,tipo_produto,texto_busca,data_analise,marketplace,arquivo_pagina,url,descricao_produto,fabricante,modelo,valor,...,qual_codigo_sch_fornecido,codigo_sch_corresponde_produto,codigo_ean_foi_fornecido,qual_codigo_ean_fornecido,codigo_ean_corresponde_produto,info_fabricante,info_tipo_produto,info_modelos_certificado,info_ean_homologados,info_modelo_ean
0,Celulares e Smartphones,smartphone,05/07/2024,Amazon,amazon_20240705_smartphone_0.pdf,https://www.amazon.com.br/Celular-Nokia-105-4G...,Celular Nokia 105 4G Preto - NK094,Nokia,NK094,375.0,...,040492213411,,Não,,,HMD Global Oy,Telefone Móvel Celular,TA-1393 | Nokia 105 4G,7908414464995,TA-1393: Nokia 105 4G
1,Celulares e Smartphones,smartphone,05/07/2024,Amazon,amazon_20240705_smartphone_1.pdf,https://www.amazon.com.br/Smartphone-Motorola-...,Smartphone Motorola Edge 30 Fusion 5G 256GB 8G...,Motorola,XT2243-1 | Galaxy,4999.0,...,081252200330,,Não,,,Motorola Mobility LLC,Telefone Móvel Celular | Transceptor de Radiaç...,XT2243-1,7892597352203\n* 7892597352210\n* 789259735260...,XT2243-1: Motorola Edge 30 Fusion
2,Celulares e Smartphones,smartphone,05/07/2024,Amazon,amazon_20240705_smartphone_2.pdf,https://www.amazon.com.br/Iphone-Apple-Preto-6...,Apple iPhone 11 (64 GB) Preto,Apple,,2789.1,...,056861901993,,Não,,,Apple Inc.,Sistemas Operando nas Faixas de RF Ultra Larga...,A2221 | iPhone 11,0194252097090\n* 0194252097106\n* 019425209740...,A2221: iPhone 11
3,Celulares e Smartphones,smartphone,05/07/2024,Amazon,amazon_20240705_smartphone_3.pdf,https://www.amazon.com.br/Apple-iPhone-15-128-...,Apple iPhone 15 (128 GB) — Azul,Apple,SUPREME-BRA,4899.0,...,127572301993,,Não,,,Apple Inc.,Transceptor de Radiação Restrita | Transceptor...,A3090 | iPhone 15,0195949035913\n* 0195949035920\n* 019594903609...,A3090: iPhone 15
4,Celulares e Smartphones,smartphone,05/07/2024,Amazon,amazon_20240705_smartphone_4.pdf,https://www.amazon.com.br/Smartphone-Infinix-B...,"Smartphone Infinix SMART 7, 3GB RAM 64GB, Câme...",Infinix,3901877 | SMART 7,719.1,...,051362315156,,Não,,,INFINIX MOBILITY LIMITED,Telefone Móvel Celular | Transceptor de Radiaç...,X6515 | X669C | Smart 7 | HOT 30i,7908068807155\n* 7908068807193\n* 790806880828...,X6515: Smart 7\n* X669C: HOT 30i


# Análise

In [215]:
map_fiscais_marketplace = {
    'Amazon': {'Atribuído para': 'werther', 'Unidade de Lotação': 'GR03'}, 
    'Mercado Livre': {'Atribuído para': 'costalo', 'Unidade de Lotação': 'UO021'}
}

df_fiscaliza = df_produtos.copy()

# título da última tarefa no Fiscaliza
# as novas terão numeração sequencial a partir desta
fiscaliza_last_issue = 00000000
format_title = lambda i: f'INSP_PACP_2024_{str(i+1+fiscaliza_last_issue).zfill(8)}'

df_fiscaliza['Projeto'] = 'Rotulagem de Dados'

df_fiscaliza['Título'] = df_fiscaliza.index.map(format_title)
df_fiscaliza['Descrição'] = df_fiscaliza.apply(lambda row: format_issue(row),axis=1)
df_fiscaliza['Tipo'] = 'Rotulagem de Dados'
df_fiscaliza['Situação'] = 'Responder pesquisa'
df_fiscaliza['Data de início'] = '08/07/2024'
df_fiscaliza['Data limite'] = '31/07/2024'

# fiscais e unidade de lotação
df_fiscaliza = df_fiscaliza.join(pd.DataFrame(df_fiscaliza['marketplace'].map(map_fiscais_marketplace).to_list()))

columns_to_issues = ['Projeto', 'Tipo', 'Situação', 'Título', 'Descrição', 
                     'Data de início', 'Data limite', 'Atribuído para', 'Unidade de Lotação', 
                     'tipo_produto', 'texto_busca', 'data_analise', 'marketplace', 
                     'fabricante', 'modelo', 'valor', 
                     'existe_campo_codigo_sch', 'codigo_sch_foi_fornecido', 'qual_codigo_sch_fornecido', 'codigo_sch_corresponde_produto', 
                     'codigo_ean_foi_fornecido', 'qual_codigo_ean_fornecido', 'codigo_ean_corresponde_produto']
df_fiscaliza = df_fiscaliza[columns_to_issues]

columns_to_rename = ['Projeto', 'Tipo', 'Situação', 'Título', 'Descrição', 
                     'Data de início', 'Data limite', 'Atribuído para', 'Unidade de Lotação',
                     'Qual o tipo do produto? ','Texto da busca','Qual foi a data da coleta?','Marketplace',
                     'Fabricante','Modelo do Produto','Valor',
                     'Existe campo código SCH?','Código SCH foi fornecido?','Qual código SCH fornecido?','Código SCH é o do produto?',
                     'Código EAN foi fornecido?','Qual código EAN fornecido?','Código EAN é o do produto?']
df_fiscaliza.columns = columns_to_rename

for collect_date in df_fiscaliza['Qual foi a data da coleta?'].unique():
    issues_file_date = datetime.strptime(collect_date,'%d/%m/%Y').strftime('%Y%m%d')
    
    issues_output_folder = Path(BASE_FOLDER)/'issues'
    issues_file_name = issues_output_folder/f'issues_{issues_file_date}.csv'
    if not issues_file_name.exists():
        df_fiscaliza[df_fiscaliza['Qual foi a data da coleta?']==collect_date].to_csv(issues_file_name,index=False,sep=";",quotechar="'")
    else:
        print(f'Issues file for collect date {collect_date} already exists. Not saved.')
        

Issues file for collect date 05/07/2024 already exists. Not saved.


In [219]:
df_fiscaliza[df_fiscaliza['Título']=='INSP_PACP_2024_00000006'].T

,5
Projeto,Rotulagem de Dados
Tipo,Rotulagem de Dados
Situação,Responder pesquisa
Título,INSP_PACP_2024_00000006
Descrição,"h1. Smartphone Samsung Galaxy M14 5G, 128 GB, ..."
Data de início,08/07/2024
Data limite,31/07/2024
Atribuído para,werther
Unidade de Lotação,GR03
Qual o tipo do produto?,Celulares e Smartphones


# Código obsoleto

In [ ]:
# file_ml = '../datasets/inspecao_ecommerce/ml_smartphone.xlsx'
# file_magalu = '../datasets/inspecao_ecommerce/magalu_smartphone.xlsx'
# file_amazon = '../datasets/inspecao_ecommerce/amazon_smartphone.xlsx'

# df_ml = pd.read_excel(file_ml,dtype='str')
# df_ml['Marketplace'] = 'Mercado Livre'

# df_amazon = pd.read_excel(file_amazon,dtype='str')
# df_amazon['Marketplace'] = 'Amazon'

# df_magalu = pd.read_excel(file_magalu,dtype='str')
# df_magalu['Marketplace'] = 'Magalu'

# df_produtos = pd.concat([df_ml, df_amazon, df_magalu])
# # rename columns
# df_produtos.columns = df_produtos.columns.map(map_columns)
# # add new columns
# df_produtos['qual_codigo_ean_fornecido'] = None


# columns_to_keep = ['tipo_produto', 'texto_busca', 'data_analise', 'marketplace',
#                    'arquivo_pagina', 'url', 'descricao_produto',
#                    'fabricante', 'modelo', 'valor', 
#                    'existe_campo_codigo_sch', 'codigo_sch_foi_fornecido', 'qual_codigo_sch_fornecido', 'codigo_sch_corresponde_produto',
#                    'codigo_ean_foi_fornecido', 'qual_codigo_ean_fornecido', 'codigo_ean_corresponde_produto']

# df_produtos = df_produtos[columns_to_keep]

# # df_produtos['Descrição'] = df_produtos.apply(lambda row: format_issue(row) ,axis=1)

# df_produtos = df_produtos.merge(df_homologados,on='qual_codigo_sch_fornecido',how='left')
# df_produtos = df_produtos.merge(df_sch_ean,on='qual_codigo_sch_fornecido',how='left')
# df_produtos['info_ean_homologados'].fillna('EAN indisponível',inplace=True)
# df_produtos['info_modelo_ean'].fillna('EAN indisponível',inplace=True)
# df_produtos = df_produtos.fillna('')

# # strip pipe caracter from the end of string
# for col in ['info_tipo_produto', 'info_modelos_certificado', 'info_ean_homologados']:
#     df_produtos[col] = df_produtos[col].apply(lambda x: x[:-3] if x[-3:]==' | ' else x)

# df_produtos.head()

In [ ]:
# # seleção da amostras
# n_samples, _ = df_fiscais_amostra.shape
# n_samples = 2
# df_fiscaliza = df_produtos.groupby('marketplace').sample(n_samples).sample(frac=1)
# df_fiscaliza = df_fiscaliza.reset_index(drop=True)

# # título da última tarefa no Fiscaliza
# # as novas terão numeração sequencial a partir desta
# fiscaliza_last_issue = 00000000
# format_title = lambda i: f'INSP_PACP_2024_{str(i+1+fiscaliza_last_issue).zfill(8)}'

# df_fiscaliza['Projeto'] = 'Rotulagem de Dados'

# df_fiscaliza['Título'] = df_fiscaliza.index.map(format_title)
# df_fiscaliza['Descrição'] = df_fiscaliza.apply(lambda row: format_issue(row),axis=1)

# # df_fiscaliza['Tipo'] = 'Fiscalização de Marketplace'
# df_fiscaliza['Tipo'] = 'Rotulagem de Dados'
# df_fiscaliza['Situação'] = 'Responder pesquisa'

# df_fiscaliza['Data de início'] = '08/07/2024'
# df_fiscaliza['Data limite'] = '31/07/2024'

# df_fiscaliza = df_fiscaliza.join(df_fiscais_amostra)
# df_fiscaliza = df_fiscaliza.merge(df_fiscais[['Login','Lotação']],left_on='Atribuído para', right_on='Login')

# # df_fiscaliza.to_parquet(FILE_SAMPLES,index=False)

# columns_to_issues = ['Projeto', 'Tipo', 'Situação', 'Título', 'Descrição', 
#                      'Data de início', 'Data limite', 'Atribuído para', 'Lotação', 
#                      'tipo_produto', 'texto_busca', 'data_analise', 'marketplace', 
#                      'fabricante', 'modelo', 'valor', 
#                      'existe_campo_codigo_sch', 'codigo_sch_foi_fornecido', 'qual_codigo_sch_fornecido', 'codigo_sch_corresponde_produto', 
#                      'codigo_ean_foi_fornecido', 'qual_codigo_ean_fornecido', 'codigo_ean_corresponde_produto']

# df_fiscaliza = df_fiscaliza[columns_to_issues]

# columns_to_rename = ['Projeto', 'Tipo', 'Situação', 'Título', 'Descrição', 
#                      'Data de início', 'Data limite', 'Atribuído para', 'Unidade de Lotação',
#                      'Qual o tipo do produto? ','Texto da busca','Qual foi a data da análise?','Marketplace',
#                      'Fabricante','Modelo do Produto','Valor',
#                      'Existe campo código SCH?','Código SCH foi fornecido?','Qual código SCH fornecido?','Código SCH é o do produto?',
#                      'Código EAN foi fornecido?','Qual código EAN fornecido?','Código EAN é o do produto?']
# df_fiscaliza.columns = columns_to_rename

# df_fiscaliza['Valor'] = df_fiscaliza['Valor'].apply(lambda x: 0 if x=='' else x).astype(float)

# df_fiscaliza